In [ ]:
import sys
import os
import pandas as pd
#from index.load_index import load_index
from models import BM25RetrievalModel, TFIDFRetrievalModel, DenseRetrievalModel, HybridRetrievalModel
from evaluation import evaluate, tune_hybrid_alpha, plot_alpha_scores

In [ ]:
# Load Indexed Documents
#sys.path.append(".")   # Adjust depending on your layout
index_dir = "tuebingen_index"  # Update if stored elsewhere
documents = load_index(index_dir)

texts = [doc["text"] for doc in documents]
urls = [doc["url"] for doc in documents]

print(f"Loaded {len(texts)} documents from index.")
# Load Retrieval Models
bm25_model = BM25RetrievalModel(texts, urls)
tfidf_model = TFIDFRetrievalModel(texts, urls)
dense_model = DenseRetrievalModel(texts, urls)
hybrid_model = HybridRetrievalModel(bm25_model, dense_model, alpha=0.5)

queries = [
    "tübingen attractions",
    "food and drinks",
    "university of tübingen history",
    "student life",
    "museums in tübingen"
]
# Evaluate Models on Queries
def display_results(model, model_name, query, top_n=5):
    print(f"\n{model_name} Results for: \"{query}\"")
    results = model.retrieve(query, top_k=top_n)
    for i, res in enumerate(results):
        print(f"{i+1}. {res['url']} (Score: {res['score']:.4f})")
        print(f"   {res['snippet'][:200]}...\n")

for query in queries:
    display_results(bm25_model, "BM25", query)
    display_results(tfidf_model, "TF-IDF", query)
    display_results(dense_model, "Dense Embedding", query)
    display_results(hybrid_model, "Hybrid Model (BM25 + Dense)", query)
    
# Compare Side-by-Side
def get_result_df(model, query, top_n=5):
    results = model.retrieve(query, top_k=top_n)
    return pd.DataFrame([{
        'Rank': i + 1,
        'URL': res['url'],
        'Score': res['score'],
        'Snippet': res['snippet'][:150]
    } for i, res in enumerate(results)])

query_idx = 0  # Choose from 0 to len(queries)-1
query = queries[query_idx]

print(f"\n\nSide-by-Side Result for Query: \"{query}\"")
bm25_df = get_result_df(bm25_model, query)
tfidf_df = get_result_df(tfidf_model, query)
dense_df = get_result_df(dense_model, query)
hybrid_df = get_result_df(hybrid_model, query)

print("\nBM25 Top Results:")
display(bm25_df)

print("\nTF-IDF Top Results:")
display(tfidf_df)

print("\nDense Embedding Top Results:")
display(dense_df)

print("\nHybrid Model Top Results:")
display(hybrid_df)

In [ ]:
print("BM25 Evaluation:")
print(evaluate(bm25_model, 'queries.txt', 'qrels.txt', k=10))

print("Dense Evaluation:")
print(evaluate(dense_model, 'queries.txt', 'qrels.txt', k=10))

print("Hybrid Evaluation:")
print(evaluate(hybrid_model, 'queries.txt', 'qrels.txt', k=10))

In [ ]:
best_alpha, scores = tune_hybrid_alpha(
    bm25_model,
    dense_model,
    queries_path='queries.txt',
    qrels_path='qrels.txt',
    k=10
)
plot_alpha_scores(scores)